In [44]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math
from statsmodels.tsa.arima.model import ARIMA

#import csvs
train = pd.read_csv("../../data/train.csv")
test = pd.read_csv("../../data/test.csv")
graph = pd.read_csv("../../data/graph.csv")

statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [45]:

days = 26

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

def predictARIMA(X, p, d, q, days):
    model = ARIMA(X, order=(p,d,q))
    model_fit = model.fit()
    start = len(X)
    forecast = model_fit.predict(start = start, end = start + days)
    return(forecast)


proj = {}
for s in states:
    a = statesdata[s]
    a = a.reset_index()
    confirmed = a['Confirmed']
    deaths = a['Deaths']
    
    X = confirmed.values
    
    forecastC = predictARIMA(X, 5,2,5, days)
    # BEST PARAMS SO FAR ARE 3,2,1
    
    Y= deaths.values

    forecastD = predictARIMA(Y, 5,2,5, days)
    # BEST PARAMS SO FAR ARE 4,2,3

    df = {'Confirmed': forecastC, 'Deaths': forecastD}
    
    proj[s] = pd.DataFrame(df)

In [46]:
order = test.loc[0:49,'Province_State']

# format submission
conf = []
dead = []
fid = 0
for i in range(days):
    for j in order:
        projection = proj[j].iloc[i]
        #print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        dead.append(int(projection['Deaths']))
        #print(fid)
        fid+=1

test['Confirmed'] = conf
test['Deaths'] = dead

In [47]:
submission = test.drop(columns=['Province_State', 'Date'])
print(submission)
submission.to_csv('team25.csv', index = False, header = True)

ForecastID  Confirmed  Deaths
0              0     127561    2194
1              1       5314      38
2              2     202320    5054
3              3      61796     812
4              4     717808   13139
...          ...        ...     ...
1295        1295     146183    2964
1296        1296      87466    2137
1297        1297      14257     314
1298        1298      92688    1276
1299        1299       4749      43

[1300 rows x 3 columns]


In [48]:
def give_mape(ground_truth_path, prediction_path):
    ground_truth = pd.read_csv(ground_truth_path)
    predictions = pd.read_csv(prediction_path)

    gt_conf = ground_truth['Confirmed']
    gt_dead = ground_truth['Deaths']

    pred_conf = predictions['Confirmed']
    pred_dead = predictions['Deaths']

    total = 0
    for i in range(1300):
        conf_error = float(pred_conf[i] - gt_conf[i]) / float(gt_conf[i])
        dead_error = float(pred_dead[i] - gt_dead[i]) / float(gt_dead[i])
        total = total + abs(conf_error) + abs(dead_error)

    total = total / 2600
    return total

print(give_mape('team25.csv', '../ValidationTester/ground_truth.csv'))

0.025302217150302873
